In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-full-cold/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[1]

26 experiments


{'commit': '514c7903a629a2daa1518c1053c4e49376b2ad6e',
 'id': '3b53532a85a31eaa5433c22e6c8c4bdbe51ebd7c',
 'name': 'fleet-soft',
 'params': {'dataset': {'path': 'bdsaglam/musique',
   'name': 'answerable',
   'split': 'validation'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.0,
   'system_prompt': 'cok/adapted.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'semi/cok.json',
   'n_shot': 2,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.6313611915597849,
  'f1': 0.7466897905218197,
  'fuzzy_match': 0.6876292925113777,
  '2hops': {'exact_match': 0.6605431309904153,
   'f1': 0.7707371389644943,
   'fuzzy_match': 0.7300319488817891},
  '3hops': {'exact_match': 0.6263157894736842,
   'f1': 0.7584340155888143,
   'fuzzy_match': 0.6815789473684211},
  '4hops': {'exact_match': 0.5506172839506173,
   'f1': 0.650312281136278,
   'fuzzy_match': 0.5679012345679012},
  'gen_token_count': {'all': {'count': 2417.0,
    'mean': 86.55978485726106,
    'std': 

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique")
    & (df["params.qa.few_shot_examples"].str.contains("semi")) 
    & (df["metrics.f1"] > 0.3)
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

20 experiments after preprocessing


In [4]:
def parse_qa_technique(row):
    base_technique = row["params.qa.system_prompt"].split("/")[0].upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [5]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.n_sc",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.n_sc,params.qa.temperature,params.run
0,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,1,0.0,"(1,)"
1,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,1,0.0,"(1,)"
2,ccot/format-thought.txt,cq.txt,semi/ccot.json,2,1,0.0,"(1,)"
3,cok/adapted.txt,cq.txt,semi/cok.json,0,1,0.0,"(1,)"
4,cok/adapted.txt,cq.txt,semi/cok.json,1,1,0.0,"(1,)"
5,cok/adapted.txt,cq.txt,semi/cok.json,2,1,0.0,"(1,)"
6,cok/adapted.txt,cq.txt,semi/cok.json,3,1,0.0,"(1,)"
7,cot/format-thought.txt,cq.txt,semi/cot.json,0,1,0.0,"(1,)"
8,cot/format-thought.txt,cq.txt,semi/cot.json,1,1,0.0,"(1,)"
9,cot/format-thought.txt,cq.txt,semi/cot.json,2,1,0.0,"(1,)"


In [6]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	cok/adapted.txt
	cot/format-thought.txt
	cte/format-triples-few.txt
	cte/format-triplets-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	semi/ccot.json
	semi/cok.json
	semi/cot.json
	semi/cte-triples.json
	semi/cte.json
	semi/direct.json
- params.qa.n_shot
	0
	1
	2
	3
- params.qa.n_sc
	1
- params.qa.temperature
	0.0
- params.run
	(1,)


In [7]:
df.to_json('exps.jsonl', orient='records', lines=True)